<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_infinite_recursion_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_infinite_recursion_ai.py

Pipeline for InfiniteRecursionAI:
1. Synthetic “infinite recursion” dataset of 6 inputs → 3 targets
2. float32 normalization & dtype consistency
3. MLP with ReLU (simple architecture)
4. Physics‐informed residual enforcing toy recursion laws
5. MC‐Dropout for uncertainty quantification
6. Training loop: AdamW, ReduceLROnPlateau, gradient clipping, NaN checks, early stopping, checkpointing
7. Visualizations: loss curves, scatter plots, uncertainty heatmap
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split

# ------------------------------------------------------------------------------
# 1. Synthetic Infinite Recursion Dataset
# ------------------------------------------------------------------------------
class InfiniteRecursionDataset(Dataset):
    def __init__(self, n_samples=6000, seed=42):
        np.random.seed(seed)
        # Inputs:
        # RIE: recursive expansion rate ∈ [0.1, 5.0]
        # FAS: fractal AGI synthesis scale ∈ [0.01, 2.0]
        # ORD: omniversal recursion depth ∈ [1, 100]
        # EIR: energy injection rate ∈ [1, 1000]
        # CXS: complexity scaling ∈ [0.1, 10.0]
        # DIT: dimensional iteration threshold ∈ [0.5, 5.0]
        RIE = np.random.uniform(0.1, 5.0,   (n_samples,1))
        FAS = np.random.uniform(0.01,2.0,   (n_samples,1))
        ORD = np.random.uniform(1, 100,     (n_samples,1))
        EIR = np.random.uniform(1, 1000,    (n_samples,1))
        CXS = np.random.uniform(0.1,10.0,   (n_samples,1))
        DIT = np.random.uniform(0.5,5.0,    (n_samples,1))

        X_raw = np.hstack([RIE, FAS, ORD, EIR, CXS, DIT]).astype(np.float64)

        # Toy targets:
        # IRS: infinite recursion stability = RIE * np.log1p(ORD) / (FAS + eps)
        # SGR: self-generation rate = FAS * np.sqrt(EIR)
        # RCS: recursive complexity scaling = CXS * DIT / (RIE + eps)
        eps = 1e-8
        IRS = RIE * np.log1p(ORD) / (FAS + eps)
        SGR = FAS * np.sqrt(EIR)
        RCS = CXS * DIT / (RIE + eps)

        Y_raw = np.hstack([IRS, SGR, RCS]).astype(np.float64)
        Y_raw += 0.01 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape)

        # Stats
        self.X_mean = X_raw.mean(axis=0)
        self.X_std  = X_raw.std(axis=0) + eps
        self.Y_mean = Y_raw.mean(axis=0)
        self.Y_std  = Y_raw.std(axis=0) + eps

        # Normalize to float32
        self.X = ((X_raw - self.X_mean) / self.X_std).astype(np.float32)
        self.Y = ((Y_raw - self.Y_mean) / self.Y_std).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.from_numpy(self.Y[idx])

# ------------------------------------------------------------------------------
# 2. InfiniteRecursionAI Model
# ------------------------------------------------------------------------------
class InfiniteRecursionAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=32, output_dim=3, p_drop=0.1):
        super().__init__()
        self.fc1  = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p_drop)
        self.fc2  = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.fc2(x)
        return x

# ------------------------------------------------------------------------------
# 3. Physics-Informed Residual Loss
# ------------------------------------------------------------------------------
def physics_residual(pred, X, stats):
    X_den = X * stats['X_std'] + stats['X_mean']
    RIE, FAS, ORD, EIR, CXS, DIT = X_den.t()
    eps = 1e-8

    IRS_t = RIE * torch.log1p(ORD) / (FAS + eps)
    SGR_t = FAS * torch.sqrt(EIR)
    RCS_t = CXS * DIT / (RIE + eps)

    Yt      = torch.stack([IRS_t, SGR_t, RCS_t], dim=1)
    Yt_norm = (Yt - stats['Y_mean']) / stats['Y_std']
    return nn.MSELoss()(pred, Yt_norm)

# ------------------------------------------------------------------------------
# 4. Combined Loss Function
# ------------------------------------------------------------------------------
def total_loss(pred, true, X, stats, lambda_phys=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, X, stats)
    return mse + lambda_phys * phys, mse, phys

# ------------------------------------------------------------------------------
# 5. MC-Dropout Uncertainty Estimation
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    arr = torch.stack(preds, dim=0)
    return arr.mean(dim=0), arr.std(dim=0)

# ------------------------------------------------------------------------------
# 6. Training Loop & Checkpointing
# ------------------------------------------------------------------------------
def train(model, train_loader, val_loader, stats, device,
          lr=1e-4, wd=1e-5, lambda_phys=1.0, epochs=100, patience=10):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )

    best_val, wait = float('inf'), 0
    history = {'train': [], 'val': []}

    for ep in range(1, epochs+1):
        model.train()
        run_tr = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss, _, _ = total_loss(pred, yb, xb, stats, lambda_phys)
            if torch.isnan(loss):
                raise RuntimeError(f"NaN loss at epoch {ep}")
            optimizer.zero_grad(); loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            run_tr += loss.item() * xb.size(0)
        train_loss = run_tr / len(train_loader.dataset)

        model.eval()
        run_va = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                l, _, _ = total_loss(pred, yb, xb, stats, lambda_phys)
                run_va += l.item() * xb.size(0)
        val_loss = run_va / len(val_loader.dataset)

        scheduler.step(val_loss)
        history['train'].append(train_loss)
        history['val'].append(val_loss)
        print(f"Epoch {ep:03d} | Train {train_loss:.4e} | Val {val_loss:.4e}")

        if val_loss < best_val - 1e-6:
            best_val, wait = val_loss, 0
            torch.save(model.state_dict(), "best_infinite_recursion_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping.")
                break

    model.load_state_dict(torch.load(
        "best_infinite_recursion_ai.pth", map_location=device))
    return history

# ------------------------------------------------------------------------------
# 7. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_history(history):
    plt.figure()
    plt.plot(history['train'], label='Train Loss')
    plt.plot(history['val'],   label='Val Loss')
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.show()

def plot_scatter(y_true, y_pred, title):
    plt.figure()
    plt.scatter(y_true, y_pred, s=6, alpha=0.5)
    mn, mx = y_true.min(), y_true.max()
    plt.plot([mn,mx],[mn,mx],'r--')
    plt.title(title); plt.show()

def plot_uncertainty(model, stats, device):
    G = 100
    RIF = np.linspace(0.1,5.0,G, dtype=np.float32)
    ORD = np.linspace(1,100, G, dtype=np.float32)
    G1, G2 = np.meshgrid(RIF, ORD)
    pts = G*G

    Xg = torch.zeros((pts,6), device=device)
    # fix FAS,EIR,CXS,DIT at mean
    for i in [1,3,4,5]:
        Xg[:, i] = stats['X_mean'][i]
    Xg[:,0] = torch.from_numpy(G1.ravel()).to(device)
    Xg[:,2] = torch.from_numpy(G2.ravel()).to(device)

    Xn = (Xg - stats['X_mean']) / stats['X_std']
    _, std = mc_dropout_predict(model, Xn, T=40)
    U = std[:,0].cpu().reshape(G1.shape)

    plt.figure(figsize=(5,4))
    plt.pcolormesh(RIF, ORD, U, shading='auto', cmap='magma')
    plt.colorbar(label="Std(Infinite Stability)")
    plt.xlabel("Recursion Rate (RIE)")
    plt.ylabel("Recursion Depth (ORD)")
    plt.title("Uncertainty Heatmap")
    plt.show()

# ------------------------------------------------------------------------------
# 8. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ds    = InfiniteRecursionDataset(n_samples=6000, seed=42)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, dtype=torch.float32, device=device),
        'X_std' : torch.tensor(ds.X_std,  dtype=torch.float32, device=device),
        'Y_mean': torch.tensor(ds.Y_mean, dtype=torch.float32, device=device),
        'Y_std' : torch.tensor(ds.Y_std,  dtype=torch.float32, device=device),
    }

    n_val = int(0.2 * len(ds))
    tr_ds, va_ds = random_split(ds, [len(ds)-n_val, n_val])
    tr_ld = DataLoader(tr_ds, batch_size=128, shuffle=True)
    va_ld = DataLoader(va_ds, batch_size=256, shuffle=False)

    model   = InfiniteRecursionAI().to(device)
    history = train(model, tr_ld, va_ld, stats, device)

    plot_history(history)

    X_all = torch.from_numpy(ds.X).to(device)
    with torch.no_grad():
        Yp_n = model(X_all).cpu().numpy()
    Yt = ds.Y * ds.Y_std + ds.Y_mean
    Yp = Yp_n * ds.Y_std + ds.Y_mean

    names = ["Recursion Stability","Self-Generation Rate","Complexity Scaling"]
    for i, nm in enumerate(names):
        plot_scatter(Yt[:,i], Yp[:,i], nm)

    plot_uncertainty(model, stats, device)